<a href="https://colab.research.google.com/github/Aqel06/Kelompok-1-KAV/blob/main/Kelompok_1_KAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/Kelompok-1-KAV
!pwd
!ls -la
!ls -la mir_chroma
!ls -la scripts

[Errno 2] No such file or directory: '/content/Kelompok-1-KAV'
/content
/content
total 16
drwxr-xr-x 1 root root 4096 Oct  1 13:39 .
drwxr-xr-x 1 root root 4096 Oct  3 12:45 ..
drwxr-xr-x 4 root root 4096 Oct  1 13:38 .config
drwxr-xr-x 1 root root 4096 Oct  1 13:39 sample_data
ls: cannot access 'mir_chroma': No such file or directory
ls: cannot access 'scripts': No such file or directory


Struktur folder dasar

In [2]:
import os
for d in ["mir_chroma", "scripts", "data", "outputs"]:
    os.makedirs(d, exist_ok=True)

with open("mir_chroma/__init__.py", "w", encoding="utf-8") as f:
    f.write("# package marker\n")

print("Siap. Folder & __init__ dibuat.")

Siap. Folder & __init__ dibuat.


1. mir_chroma/feature.py

In [3]:
%%writefile mir_chroma/feature.py
from __future__ import annotations
import numpy as np
import librosa

def _normalize_chroma(C: np.ndarray, norm: str = "l1") -> np.ndarray:
    if norm == "l1":
        denom = np.maximum(np.sum(np.abs(C), axis=0, keepdims=True), 1e-8)
        return C / denom
    if norm == "l2":
        denom = np.maximum(np.sqrt(np.sum(C**2, axis=0, keepdims=True)), 1e-8)
        return C / denom
    return C

def chroma_stft(y: np.ndarray, sr: int, hop: int = 2048, n_fft: int = 4096) -> np.ndarray:
    C = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop, n_fft=n_fft)
    return _normalize_chroma(C, "l1")

def chroma_cqt(y: np.ndarray, sr: int, hop: int = 2048, bins_per_octave: int = 12) -> np.ndarray:
    C = librosa.feature.chroma_cqt(y=y, sr=sr, hop_length=hop, bins_per_octave=bins_per_octave)
    return _normalize_chroma(C, "l1")

def chroma_cens(y: np.ndarray, sr: int, hop: int = 4096, win: int = 41) -> np.ndarray:
    C = librosa.feature.chroma_cens(y=y, sr=sr, hop_length=hop, win_len=win)
    return _normalize_chroma(C, "l1")

def beat_sync(C: np.ndarray, y: np.ndarray, sr: int, hop: int):
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr, hop_length=hop, units="frames")
    if len(beats) < 2:
        beats = np.arange(0, C.shape[1], 10)
        if beats[-1] != C.shape[1]-1:
            beats = np.append(beats, C.shape[1]-1)
    C_sync = librosa.util.sync(C, beats, aggregate=np.mean)
    return C_sync, beats

def rotate_chroma(C: np.ndarray, k: int) -> np.ndarray:
    return np.roll(C, shift=k, axis=0)

Writing mir_chroma/feature.py


2. mir_chroma/templates.py

In [4]:
%%writefile mir_chroma/templates.py
from __future__ import annotations
import numpy as np

PC = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]

def rotate(vec12: np.ndarray, k: int) -> np.ndarray:
    return np.roll(vec12, k)

MAJOR_SCALE = np.array([1,0,1,0,1,1,0,1,0,1,0,1], dtype=float)
NAT_MINOR_SCALE = np.array([1,0,1,1,0,1,0,1,1,0,1,0], dtype=float)

def all_scale_templates() -> dict[str, np.ndarray]:
    temps = {}
    for k in range(12):
        temps[f"{PC[k]}:maj_scale"] = rotate(MAJOR_SCALE, k)
        temps[f"{PC[k]}:min_scale"] = rotate(NAT_MINOR_SCALE, k)
    return temps

KS_MAJOR = np.array([6.35,2.23,3.48,2.33,4.38,4.09,2.52,5.19,2.39,3.66,2.29,2.88], dtype=float)
KS_MINOR = np.array([6.33,2.68,3.52,5.38,2.60,3.53,2.54,4.75,3.98,2.69,3.34,3.17], dtype=float)

def all_key_profiles() -> dict[str, np.ndarray]:
    profiles = {}
    for k in range(12):
        profiles[f"{PC[k]}:maj"] = rotate(KS_MAJOR, k) / KS_MAJOR.sum()
        profiles[f"{PC[k]}:min"] = rotate(KS_MINOR, k) / KS_MINOR.sum()
    return profiles

def chord_template(quality: str = "maj", n_harm: int = 3) -> np.ndarray:
    tones = [0, 4, 7] if quality == "maj" else [0, 3, 7]
    T = np.zeros(12, dtype=float)
    for h in range(1, n_harm+1):
        w = 1.0 / h
        for t in tones:
            T[(t * h) % 12] += w
    return T / (T.max() + 1e-9)

def all_chord_templates(n_harm: int = 3) -> dict[str, np.ndarray]:
    temps = {}
    for k in range(12):
        temps[f"{PC[k]}:maj"] = rotate(chord_template("maj", n_harm), k)
        temps[f"{PC[k]}:min"] = rotate(chord_template("min", n_harm), k)
    return temps

Writing mir_chroma/templates.py


3. mir_chroma/chord_recognizer.py

In [5]:
%%writefile mir_chroma/chord_recognizer.py
from __future__ import annotations
import numpy as np
import librosa
from typing import List, Tuple, Dict
from .feature import chroma_cqt, chroma_stft, chroma_cens, _normalize_chroma
from .templates import all_chord_templates

def framewise_template_match(C: np.ndarray, templates: Dict[str, np.ndarray]) -> Tuple[List[str], np.ndarray]:
    labels, scores = [], []
    keys = list(templates.keys())
    Tmat = np.stack([templates[k] for k in keys], axis=1)  # (12, K)
    Tmat = _normalize_chroma(Tmat, "l2")
    for t in range(C.shape[1]):
        v = C[:, t]
        sim = (v[:, None] * Tmat).sum(axis=0) / (np.linalg.norm(v) * np.linalg.norm(Tmat, axis=0) + 1e-9)
        i = int(np.argmax(sim))
        labels.append(keys[i]); scores.append(sim[i])
    return labels, np.array(scores)

def viterbi_smooth(labels: List[str], self_prob: float = 0.97) -> List[str]:
    uniq = sorted(set(labels))
    idx = {s:i for i,s in enumerate(uniq)}
    nS, T = len(uniq), len(labels)
    import math
    log_self = math.log(self_prob)
    log_jump = math.log((1 - self_prob) / max(nS-1, 1))
    dp = np.full((nS, T), -np.inf)
    back = np.full((nS, T), -1, dtype=int)
    dp[:, 0] = 0.0
    for t in range(1, T):
        for s in range(nS):
            best, arg = -np.inf, 0
            for sp in range(nS):
                add = log_self if s == sp else log_jump
                val = dp[sp, t-1] + add
                if val > best:
                    best, arg = val, sp
            dp[s, t] = best + (0.0 if uniq[s] == labels[t] else -1.0)
            back[s, t] = arg
    end = idx[labels[-1]]
    path = [end]
    for t in range(T-1, 0, -1):
        path.append(back[path[-1], t])
    path = path[::-1]
    return [uniq[s] for s in path]

def recognize_chords(y: np.ndarray, sr: int, algo: str = "cqt", hop: int = 2048, n_harm: int = 3, smooth: bool = True):
    if algo == "cqt":
        C = chroma_cqt(y, sr, hop)
    elif algo == "stft":
        C = chroma_stft(y, sr, hop)
    elif algo == "cens":
        C = chroma_cens(y, sr, hop)
    else:
        raise ValueError("algo must be one of: cqt, stft, cens")
    temps = all_chord_templates(n_harm=n_harm)
    labels, scores = framewise_template_match(C, temps)
    if smooth:
        labels = viterbi_smooth(labels, self_prob=0.97)
    times = librosa.frames_to_time(np.arange(C.shape[1]), sr=sr, hop_length=hop)
    return times, labels, scores

Writing mir_chroma/chord_recognizer.py


4. mir_chroma/key_detection.py

In [6]:
%%writefile mir_chroma/key_detection.py
from __future__ import annotations
import numpy as np
import librosa
from typing import Tuple
from .feature import chroma_cqt, chroma_stft, chroma_cens
from .templates import all_key_profiles

def detect_key(y: np.ndarray, sr: int, algo: str = "stft", hop: int = 2048) -> Tuple[str, float]:
    if algo == "cqt":
        C = chroma_cqt(y, sr, hop)
    elif algo == "stft":
        C = chroma_stft(y, sr, hop)
    elif algo == "cens":
        C = chroma_cens(y, sr, hop)
    else:
        raise ValueError("algo must be one of: cqt, stft, cens")
    g = np.mean(C, axis=1); g = g / (np.linalg.norm(g) + 1e-9)
    profiles = all_key_profiles()
    best_key, best_score = None, -np.inf
    for k, p in profiles.items():
        p = p / (np.linalg.norm(p) + 1e-9)
        s = float(np.dot(g, p))
        if s > best_score:
            best_key, best_score = k, s
    return best_key, best_score

Writing mir_chroma/key_detection.py


5. mir_chroma/cover_id.py

In [7]:
%%writefile mir_chroma/cover_id.py
from __future__ import annotations
import numpy as np
import librosa
from .feature import chroma_cqt, chroma_stft, chroma_cens, beat_sync, rotate_chroma

def _cosine_cs(X: np.ndarray, Y: np.ndarray) -> np.ndarray:
    Xn = X / (np.linalg.norm(X, axis=0, keepdims=True) + 1e-9)
    Yn = Y / (np.linalg.norm(Y, axis=0, keepdims=True) + 1e-9)
    return np.dot(Xn.T, Yn)  # (T1 x T2)

def _dtw_distance(S: np.ndarray, band: int | None = None) -> float:
    C = 1.0 - S
    N, M = C.shape
    D = np.full((N+1, M+1), np.inf); D[0,0] = 0.0
    for i in range(1, N+1):
        j_start = 1 if band is None else max(1, i - band)
        j_end   = M+1 if band is None else min(M+1, i + band + 1)
        for j in range(j_start, j_end):
            D[i,j] = C[i-1,j-1] + min(D[i-1,j], D[i,j-1], D[i-1,j-1])
    return float(D[N,M] / (N + M))

def cover_similarity(y1: np.ndarray, sr1: int, y2: np.ndarray, sr2: int, algo: str = "cens", hop: int = 4096):
    y1 = librosa.to_mono(y1); y2 = librosa.to_mono(y2)
    if sr1 != sr2:
        y2 = librosa.resample(y2, orig_sr=sr2, target_sr=sr1); sr2 = sr1
    if algo == "cqt":
        C1 = chroma_cqt(y1, sr1, hop); C2 = chroma_cqt(y2, sr2, hop)
    elif algo == "stft":
        C1 = chroma_stft(y1, sr1, hop); C2 = chroma_stft(y2, sr2, hop)
    elif algo == "cens":
        C1 = chroma_cens(y1, sr1, hop); C2 = chroma_cens(y2, sr2, hop)
    else:
        raise ValueError("algo must be one of: cqt, stft, cens")

    C1b, _ = beat_sync(C1, y1, sr1, hop)
    C2b, _ = beat_sync(C2, y2, sr2, hop)

    best_k, best_d = 0, np.inf
    for k in range(12):
        S = _cosine_cs(C1b, rotate_chroma(C2b, k))
        d = _dtw_distance(S, band=None)
        if d < best_d:
            best_d, best_k = d, k
    return best_k, best_d

Writing mir_chroma/cover_id.py


6. scripts/demo_chord.py

In [8]:
%%writefile scripts/demo_chord.py
import argparse, os, numpy as np, librosa, matplotlib.pyplot as plt, csv
from mir_chroma.chord_recognizer import recognize_chords

p = argparse.ArgumentParser()
p.add_argument("audio")
p.add_argument("--algo", choices=["cqt","stft","cens"], default="cqt")
p.add_argument("--hop", type=int, default=2048)
p.add_argument("--harm", type=int, default=3)
p.add_argument("--no-smooth", action="store_true")
p.add_argument("--outdir", default="outputs")
args = p.parse_args()

y, sr = librosa.load(args.audio, sr=None, mono=True)
times, labels, scores = recognize_chords(
    y, sr, algo=args.algo, hop=args.hop, n_harm=args.harm, smooth=(not args.no_smooth)
)

os.makedirs(args.outdir, exist_ok=True)
stem = os.path.splitext(os.path.basename(args.audio))[0]
csv_path = os.path.join(args.outdir, f"{stem}_chords.csv")
png_path = os.path.join(args.outdir, f"{stem}_chords.png")

with open(csv_path, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f); w.writerow(["time_sec","chord","score"])
    for t, lab, sc in zip(times, labels, scores):
        w.writerow([f"{t:.3f}", lab, f"{sc:.4f}"])

plt.figure(figsize=(12,3)); plt.plot(times, scores, lw=1.5)
plt.yticks([0,0.25,0.5,0.75,1.0]); plt.xlabel("Time (s)"); plt.ylabel("Template match")
for i in range(0, len(times), max(1, len(times)//20)):
    plt.text(times[i], 0.05, labels[i], rotation=90, fontsize=8, va="bottom")
plt.tight_layout(); plt.savefig(png_path, dpi=160)
print("Saved:", csv_path, png_path)

Writing scripts/demo_chord.py


7. scripts/demo_key.py

In [9]:
%%writefile scripts/demo_key.py
import argparse, librosa
from mir_chroma.key_detection import detect_key

p = argparse.ArgumentParser()
p.add_argument("audio")
p.add_argument("--algo", choices=["cqt","stft","cens"], default="stft")
p.add_argument("--hop", type=int, default=2048)
args = p.parse_args()

y, sr = librosa.load(args.audio, sr=None, mono=True)
key, score = detect_key(y, sr, algo=args.algo, hop=args.hop)
print(f"Estimated key: {key} (score={score:.4f})")

Writing scripts/demo_key.py


8. scripts/demo_cover.py

In [10]:
%%writefile scripts/demo_key.py
import argparse, librosa
from mir_chroma.key_detection import detect_key

p = argparse.ArgumentParser()
p.add_argument("audio")
p.add_argument("--algo", choices=["cqt","stft","cens"], default="stft")
p.add_argument("--hop", type=int, default=2048)
args = p.parse_args()

y, sr = librosa.load(args.audio, sr=None, mono=True)
key, score = detect_key(y, sr, algo=args.algo, hop=args.hop)
print(f"Estimated key: {key} (score={score:.4f})")

Overwriting scripts/demo_key.py


9. requirements.txt

In [11]:
%%writefile requirements.txt
librosa>=0.10.1
numpy>=1.23
scipy>=1.9
matplotlib>=3.7
soundfile>=0.12
tqdm>=4.66

Writing requirements.txt


10. Install dependensi

In [12]:
!pip install -U pip
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [13]:
# === Generate audio sintetis: progresi Cmaj (I–IV–V–I) + cover versi transpose ===
import numpy as np, soundfile as sf, librosa, os

sr = 22050
dur = 2.0  # durasi tiap chord (detik)

def sine_tone(freq, duration, sr):
    t = np.linspace(0, duration, int(sr*duration), endpoint=False)
    return 0.25*np.sin(2*np.pi*freq*t)

def midi_to_hz(m):
    return 440.0 * (2.0 ** ((m - 69) / 12.0))

def chord_signal(root_midi, quality="maj", duration=2.0, sr=22050):
    # Triad mayor/minor sederhana (tanpa overtone)
    intervals = [0,4,7] if quality=="maj" else [0,3,7]
    y = np.zeros(int(sr*duration))
    for iv in intervals:
        y += sine_tone(midi_to_hz(root_midi + iv), duration, sr)
    y /= len(intervals)  # normalisasi biar ga clipping
    # fade in/out ringan biar halus
    n = len(y)
    fade = int(0.01*sr)
    y[:fade] *= np.linspace(0,1,fade)
    y[-fade:] *= np.linspace(1,0,fade)
    return y

# Progresi I–IV–V–I di C mayor: C(60), F(65), G(67), C(60)
prog = [60, 65, 67, 60]
y = np.concatenate([chord_signal(m, "maj", dur, sr) for m in prog])

os.makedirs("data", exist_ok=True)
ref_path = "data/chords_Cmaj.wav"
sf.write(ref_path, y, sr)

# Buat cover: transpose +2 semitone dan tempo +5%
y_cov = librosa.effects.pitch_shift(y, sr=sr, n_steps=2)
y_cov = librosa.effects.time_stretch(y_cov, rate=1.05)
cov_path = "data/chords_Cmaj_cover_shift2.wav"
sf.write(cov_path, y_cov, sr)

print("Saved:", ref_path, "and", cov_path)

Saved: data/chords_Cmaj.wav and data/chords_Cmaj_cover_shift2.wav


In [14]:
# CHORD RECOGNITION (pakai audio sintetis contoh)
!python scripts/demo_chord.py data/chords_Cmaj.wav --algo cqt --hop 2048

# KEY DETECTION (pakai audio sintetis contoh)
!python scripts/demo_key.py data/chords_Cmaj.wav --algo stft

# COVER ID (ref vs cover transpose +2)
!python scripts/demo_cover.py data/chords_Cmaj.wav data/chords_Cmaj_cover_shift2.wav --algo cens --hop 4096

Traceback (most recent call last):
  File "/content/scripts/demo_chord.py", line 2, in <module>
    from mir_chroma.chord_recognizer import recognize_chords
ModuleNotFoundError: No module named 'mir_chroma'
Traceback (most recent call last):
  File "/content/scripts/demo_key.py", line 2, in <module>
    from mir_chroma.key_detection import detect_key
ModuleNotFoundError: No module named 'mir_chroma'
python3: can't open file '/content/scripts/demo_cover.py': [Errno 2] No such file or directory
